Before you begin, execute this cell to import numpy and packages from the D-Wave Ocean suite, and all necessary functions for the gate-model framework you are going to use, whether that is the Forest SDK or Qiskit. In the case of Forest SDK, it also starts the qvm and quilc servers.

In [1]:
%run -i "assignment_helper.py"
%matplotlib inline

Available frameworks:
Forest SDK
Qiskit
D-Wave Ocean


**Exercise 1** (1 point). We want to solve the equation $Ax=b$ with $A = \begin{bmatrix}1 & 0 \\0 & -1 \\ \end{bmatrix}$ and $b =\begin{bmatrix} 0 \\ 1 \\ \end{bmatrix}$ with quantum matrix inversion. We will encode $A$ in the unitary matrix $U=e^{iAt_0}$ with $t_0=\pi/2$, and $b$ in a register. With the ancilla (qubit 0), the eigenvalue registers (or the ancilla qubits of phase estimation, qubits 1 and 2), and the eigenstate, you will need a total of four qubits and one classical register for post-selection. Prepare the superposition in the eigenvalue register and the vector $b$. Place your solution in an object called `hhl`.

In [2]:
###
q = QuantumRegister(4, 'q')
c = ClassicalRegister(4, 'c')
hhl = QuantumCircuit(q, c)
hhl.h(q[1])
hhl.h(q[2])
hhl.x(q[3])
###


In [3]:
amplitudes = get_amplitudes(hhl)
assert np.allclose(amplitudes, np.array([0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j,
       0. +0.j, 0.5+0.j, 0. +0.j, 0.5+0.j, 0. +0.j, 0.5+0.j, 0. +0.j, 0.5+0.j, 0. +0.j]))

**Exercise 2** (2 points). Start the quantum phase estimation by applying $C-U^{2^0}$ and $C-U^{2^1}$. Extend the circuit with the two appropriate gates.

In [4]:
###
hhl.cz(q[2], q[3])
###


In [5]:
amplitudes = get_amplitudes(hhl)
assert np.allclose(amplitudes, np.array([ 0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,
                                          0. +0.j,  0. +0.j,  0.5+0.j,  0. +0.j,  0.5+0.j,  0. +0.j,
                                         -0.5+0.j,  0. +0.j, -0.5+0.j,  0. +0.j]))

**Exercise 3** (1 point). Apply the quantum inverse Fourier transformation. Don't forget the swap just before the transformation.

In [6]:
###
hhl.swap(q[1], q[2])
hhl.h(q[2])
hhl.cu1(-np.pi / 2, q[1], q[2])
hhl.h(q[1])
###


In [7]:
amplitudes = get_amplitudes(hhl)
assert np.allclose(amplitudes, np.array([0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
                                         0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]))

**Exercise 4** (1 point). After this step, swap the eigenvalue registers again (this time to perform the inversion) and apply the controlled rotation. Use the same angle as in the lecture notebook.

In [8]:
###
hhl.swap(q[1], q[2])
hhl.cu3(0.392699, 0, 0, q[1], q[0])  
hhl.cu3(0.19634955, 0, 0, q[2], q[0])
###


In [9]:
amplitudes = get_amplitudes(hhl)
assert np.allclose(amplitudes, np.array([0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
                                         0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
                                         0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
                                         0.99518473+0.j, 0.09801714+0.j, 0.        +0.j, 0.        +0.j]))

**Exercise 5** (2 points). Uncompute the eigenvalue register

In [10]:
###
hhl.swap(q[1], q[2])
hhl.h(q[1])
hhl.cu1(np.pi / 2, q[1], q[2])
hhl.h(q[2])
hhl.swap(q[1], q[2])
hhl.cz(q[2], q[3])
hhl.h(q[1])
hhl.h(q[2])
###


In [11]:
amplitudes

array([0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
       0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
       0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
       0.99518473+0.j, 0.09801714+0.j, 0.        +0.j, 0.        +0.j])

In [12]:
amplitudes = get_amplitudes(hhl)
assert np.allclose(amplitudes, np.array([0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
                                         0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
                                         0.99518473+0.j, 0.09801714+0.j, 0.        +0.j, 0.        +0.j,
                                         0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j]))

At this point, if we measure 1 in the ancilla (qubit 0), the state will be proportional to $A^{-1}b = \sum_j \beta_j\lambda_j^{-1}\left|u_j\right\rangle=0 \frac{1}{1} |0\rangle + 1 \frac{1}{-1} |1\rangle=-|1\rangle$.

In [14]:
from qiskit import BasicAer

hhl.measure(q, c)

backend = BasicAer.get_backend('qasm_simulator') 
result = execute(hhl, backend, shots=100).result()
counts  = result.get_counts(hhl)
print(counts)

{'0010': 100}
